In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('dark_background')

%load_ext autoreload

%autoreload 2

In [2]:
root_folder = Path.cwd().parent.parent
data_folder = Path(root_folder).joinpath('data', 'external')

In [3]:
from src.data.task_1_food_security.dataset import Dataset

In [4]:
ds = Dataset(root_folder=root_folder)

ds.prepare_dataset_array()

In [5]:
import tensorflow as tf
from keras.layers import LSTM, Conv2D, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import numpy

Using TensorFlow backend.


In [ ]:
from keras.losses import MeanSquaredError


In [ ]:



def create_new_model(optimizer="adam", lr=None, batch_norm=False):
    model = Sequential()
    model.add(Conv2D())
    model.add(LSTM(126, input_shape=(36, 11)))
    if batch_norm:
        model.add(BatchNormalization())
    # model.add(Conv2D())
    model.add(Dense(1, activation='linear'))
    if optimizer == "adam":
        if lr is not None:
            opt = Adam(lr=lr)
        else:
            opt = Adam()
    elif optimizer == "sgd":
        if lr is not None:
            opt = SGD(lr=lr)
        else:
            opt = SGD()
    elif optimizer == "rmsprop":
        if lr is not None:
            opt = RMSprop(lr=lr)
        else:
            opt = RMSprop()
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=opt,
              metrics=[tf.keras.losses.MeanSquaredError()])
    return model

In [34]:
x = ds.X_train
y = ds.y_train_sev


# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasRegressor(build_fn=create_new_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['sgd', 'adam', 'rmsprop']
lr = [0.01, 0.005, 0.001]
epochs = [100]
batches = [100]
batch_norm = [True, False]

param_grid = dict(lr = lr, epochs=epochs, batch_norm=batch_norm) # dict(optimizer=optimizers, epochs=epochs, batch_size=batches, batch_norm=batch_norm)
model = KerasRegressor(build_fn=create_new_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    verbose=2,
                    scoring='neg_mean_squared_error')

In [35]:
grid_result = grid.fit(x, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] batch_norm=True, epochs=100, lr=0.01 ............................
[CV] ............. batch_norm=True, epochs=100, lr=0.01, total=  10.6s
[CV] batch_norm=True, epochs=100, lr=0.01 ............................
[CV] ............. batch_norm=True, epochs=100, lr=0.01, total=  12.8s
[CV] batch_norm=True, epochs=100, lr=0.01 ............................
[CV] ............. batch_norm=True, epochs=100, lr=0.01, total=  14.0s
[CV] batch_norm=True, epochs=100, lr=0.01 ............................
[CV] ............. batch_norm=True, epochs=100, lr=0.01, total=  14.2s
[CV] batch_norm=True, epochs=100, lr=0.01 ............................
[CV] ............. batch_norm=True, epochs=100, lr=0.01, total=  15.6s
[CV] batch_norm=True, epochs=100, lr=0.005 ...........................
[CV] ............ batch_norm=True, epochs=100, lr=0.005, total=  17.1s
[CV] batch_norm=True, epochs=100, lr=0.005 ...........................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  8.3min finished


In [31]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.271737 using {}
-0.271737 (0.090563) with: {}


In [18]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.256165 using {}
-0.256165 (0.100934) with: {}


In [36]:

grid_result

GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7fe5f0e6f890>,
             param_grid={'batch_norm': [True, False], 'epochs': [100],
                         'lr': [0.01, 0.005, 0.001]},
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
grid_result.predict